# --Julia High Performance Computing--

-------------------------------
1 **Features**

* fast and dynamic

> **JIT**(Just In Time) compiled rather than interpreted one:

> Just-in-Time compilation is a technique in which the code in a high-level language is converted to machine code for execution on the CPU at runtime. This is in contrast to interpreted languages, whose runtime executes the source language directly. This usually has a significantly higher overhead. On the other hand, **Ahead of Time (AOT)** compilation refers to the technique of converting source language into machine code as a separate step prior to running the code. In this case, the converted machine code can usually be saved to disk as an executable file

> This compilation infrastructure is build on top of **LLVM**(Low Level Virtual Machine)

> The Julia runtime generates LLVM **Intermediate Representation(IR)** and hands it over to LLVM's JIT compiler, which in turn generates machine code that is executed on the CPU

* type system(key ingredient for HPC)

> Domain theory by **Dana Scott**

> Dataflow-based algorithm

* multiple dispatch as its defining paradigm

* low-overhead

* currently a single-threaded language (although it does perform asynchronous I/O)

## Reference
Julia High Performance by Avik Sengupta(April 2016)

> Example:

$$\sum_{n=1}^{1000} \frac{1}{n^2}$$

In [1]:
# simple unoptimized Julia code
function pisum()
    sum = 0.0
    # run 500 times
    for j = 1:500
        # summation of finite series
        sum = 0.0
        for k = 1:1000
            sum += 1.0 / (k * k)
        end
    end
    sum
end
pisum()

1.6439345666815615

-------------------------------
2 **Analyzing Julia Performance**

* Timing Julia functions

In [2]:
# print the wall-clock time, not CPU time
tic()
sqrt(rand(1000))
toc()

elapsed time: 0.325793046 seconds


In [3]:
# macro
@time sqrt(rand(1000));

s = 0
@time for i = 1:1000
    s = s + sqrt(i)
end

  0.000023 seconds (18 allocations: 16.641 KB)
  0.015868 seconds (2.30 k allocations: 47.691 KB)


In [4]:
# an enhanced version of @time
@timev sqrt(rand(1000));

  0.000053 seconds (9 allocations: 16.078 KB)
elapsed time (ns): 53405
bytes allocated:   16464
pool allocs:       7
non-pool GC allocs:2


* Profiling Julia functions

> built-in: sampling profiler

> GUI: **Pkg.add("ProfileView")**; and then simply call **ProfileView.view()** instead of **Profile.print()**

> measure which lines of code contribute the most to the total execution time of a codebase

In [5]:
using Base.Profile
# the profiler stores its samples in memory to be viewed later
Profile.clear()
function testfunc()
    # 1000 sets of 10000 random numbers
    x = rand(10000, 1000)
    y = std(x, 1)
    return y
end
# sampling interval, the default delay is 1 millisecond on Linux
Profile.init(delay = 0.01)
# collect profile samples
@profile testfunc()

Profile.print()

75 ./task.jl:360; (::IJulia.##13#19)()
 75 ...IJulia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  75 ...rc/execute_request.jl:157; execute_request(::ZMQ.Socket, ::I...
   75 ./loading.jl:441; include_string(::String, ::String)
    74 ./<missing>:?; anonymous
     74 ./profile.jl:16; macro expansion;
      25 ./In[5]:6; testfunc;


* Tracking detailed memory allocation

> **julia**> track -allocation=user

> Julia will track all the memory used, which will be written to .mem files when Julia exits.

> When running Julia code, the compiler will compile user code at runtime. Once again, we do not want to measure the memory allocation due to the compiler. To achieve this,  rst run the code under measurement once, after starting the Julia process. Then run the **Profile.clear_malloc_data()** function to restart the allocation measurement counters. Finally, run the code under measurement once again, and then exit the process. This way, we will
get the most accurate memory measurements.

* Accurate benchmarking

> Pkg.clone("https://github.com/johnmyleswhite/Benchmarks.jl.git")

> The output will show the mean time taken to run the code, but with statistically accurate upper and lower bounds. These statistics are computed using an ordinary least squares fit of the measured execution time to estimate the expected distribution

In [6]:
using Benchmarks
# Unlike @time however, this macro can only be used in front of function calls, rather than any expression
@benchmark sqrt(rand(1000))

================ Benchmark Results ========================
     Time per evaluation: 9.72 μs [9.04 μs, 10.40 μs]
Proportion of time in GC: 1.73% [0.93%, 2.54%]
        Memory allocated: 7.97 kb
   Number of allocations: 3 allocations
       Number of samples: 5901
   Number of evaluations: 554601
         R² of OLS model: 0.554
 Time spent benchmarking: 10.83 s


-------------------------------
3 **Types**

* The Julia type system

> Function is an abstract operation represented by a name

> Methods are the individual implementations for specific types

> Dispatch is the process of selecting a function to be executed at runtime

> Dispatch is a **runtime** process, while method overloading is a **compile-time** concept

> Multiple dispatch is the method of determining the function to be called based on the types of all the parameters of the function

> **Abstract** types cannot have any instantiated values. In other words, they can only be the nodes of the type hierarchy, not its leaves. They represent sets of related types. Another special type is the **Void** type. This type has a single instance defined named **nothing**.

> **Composite** types in Julia are collections of named fields. They are equivalent to a struct in C and can be thought of as roughly equivalent to a class without behavior in object-oriented languages

> By default, the fields of a composite type can be changed at any time. In cases where this is undesirable, an immutable type can be declared using the **immutable** keyword.

> Type parameter is analogous to generic or template programming in other languages

> Type inference: based on **forward dataflow analysis**, not an implementation of the famous Hindley-Milnor algorithm using unification, which is used in the ML family of languages

In [7]:
# declare type of function argument
foo(x::Integer) = "an integer"
foo(x::ASCIIString) = "a string"

# declare type of local variable
function bar(a, b)
    x::Int64 = 0
    y = a + b
    return y
end

# dispatch on type of argument
foo(1)
foo("1")

  likely near In[7]:3


"a string"

In [8]:
# multiple dispatch
sumsqr(x, y) = x^2 + y^2
sumsqr(1, 2)
sumsqr(1.5, 2.5)
sumsqr(1 + 2im, 2 + 3im)

-8 + 16im

In [1]:
# abstract types

# supertype
abstract Number   #  <: Any
abstract Real     <: Number
abstract AbstractFloat <: Real

# leaf node
# Float64  <: AbstractFloat
# Float32  <: AbstractFloat

abstract Integer  <: Real
abstract Signed   <: Integer
abstract Unsigned <: Integer

# leaf node
# Int64    <: Signed
# Int32    <: Signed
# UInt64    <: Unsigned
# UInt32    <: Unsigned
#  abstract None   <: All above

type Pixel
    x::Int64
    y::Int64
    color::Int64
end
p = Pixel(5, 5, 100)
p.x = 10
p.x

immutable IPixel
    x::Int64
    y::Int64
    color::Int64
end
p = IPixel(5, 5, 100)
# p.x = 10

immutable Point
    x
    y
end

immutable ConcretePoint
    x::Float64
    y::Float64
end

immutable PointWithAbstract
    x::AbstractFloat
    y::AbstractFloat
end

function ParametricPoint{T <: AbstractFloat}
    x::T
    y::T
end

# T is an integer, hexadecimal string or an RGB type 
# a good example: Array{T,N} where T denotes type of elements and N denotes #(dimension)
type ParamPixel{T}
    x::Int64
    y::Int64
    color::T
end
p = ParamPixel{Int64}(5, 5, 10)
p.color

# type inference
[x for x = 1:5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

* Type-stability

> the type of the return value of a function is dependent only on the types of its arguments and not the values

In [4]:
# The return type of the Trunc function, in this case, depends on the value of the input and not just its type.
# The type of the argument for both the preceding invocations is Float64. 
# However, if the value of the input is less than zero, the type of the return is Int64. 
# On the other hand, if the input is value is zero or greater, then the type of the output is Float64. 
# This makes the function type-unstable
function Trunc(x)
    if x < 0
        return 0
    else
        return x
    end
end

Trunc(-1)
Trunc(2.5)
typeof(Trunc(2.5))
typeof(Trunc(-2.5))

Int64

In [6]:
# fixed type-stable code
function Trunc_fixed(x)
    if x < 0
        # here we can also use zero(x) function
        if typeof(x) == Float64
            return 0.0
        elseif typeof(x) == Float32
            return Float32(0.0)
        elseif typeof(x) == Int64
            return 0
        end
    else
        return x
    end
end

Trunc_fixed(-1)
Trunc_fixed(2.5)
typeof(Trunc_fixed(2.5))
typeof(Trunc_fixed(-2.5))

Float64

In [7]:
using Benchmarks
@benchmark Trunc(2.5)

================ Benchmark Results ========================
     Time per evaluation: 11.07 ns [10.86 ns, 11.28 ns]
Proportion of time in GC: 0.30% [0.06%, 0.53%]
        Memory allocated: 16.00 bytes
   Number of allocations: 1 allocations
       Number of samples: 11501
   Number of evaluations: 115133501
         R² of OLS model: 0.894
 Time spent benchmarking: 10.09 s


In [8]:
# the type-stable version does not allocate any memory, 
# while the type-unstable version does allocate quite a lot of memory
@benchmark Trunc_fixed(2.5)

================ Benchmark Results ========================
     Time per evaluation: 3.35 ns [3.29 ns, 3.41 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 4801
   Number of evaluations: 194401
         R² of OLS model: 0.955
 Time spent benchmarking: 3.65 s


In [9]:
# enables us to view the types inferred by the compiler, 
# thereby identifying any type instability in our code. 
# The output is the lowered, type-inferred AST structure

@code_warntype Trunc(2.5)

Variables:
  #self#::#Trunc
  x::Float64
  fy::Float64

Body:
  begin 
      # meta: location float.jl < 332
      fy::Float64 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      # meta: pop location
      unless (Base.box)(Base.Bool,(Base.or_int)((Base.lt_float)(x::Float64,fy::Float64)::Bool,(Base.box)(Base.Bool,(Base.and_int)((Base.box)(Base.Bool,(Base.and_int)((Base.eq_float)(x::Float64,fy::Float64)::Bool,(Base.lt_float)(fy::Float64,9.223372036854776e18)::Bool)),(Base.slt_int)((Base.box)(Int64,(Base.fptosi)(Int64,fy::Float64)),0)::Bool)))) goto 7 # line 3:
      return 0
      7:  # line 5:
      return x::Float64
  end::Union{Float64,Int64}


In [10]:
@code_warntype Trunc_fixed(2.5)

Variables:
  #self#::#Trunc_fixed
  x::Float64
  fy::Float64

Body:
  begin 
      # meta: location float.jl < 332
      fy::Float64 = (Base.box)(Float64,(Base.sitofp)(Float64,0))
      # meta: pop location
      unless (Base.box)(Base.Bool,(Base.or_int)((Base.lt_float)(x::Float64,fy::Float64)::Bool,(Base.box)(Base.Bool,(Base.and_int)((Base.box)(Base.Bool,(Base.and_int)((Base.eq_float)(x::Float64,fy::Float64)::Bool,(Base.lt_float)(fy::Float64,9.223372036854776e18)::Bool)),(Base.slt_int)((Base.box)(Int64,(Base.fptosi)(Int64,fy::Float64)),0)::Bool)))) goto 13 # line 5:
      $(QuoteNode(true)) # line 6:
      return 0.0 # line 7: # line 8: # line 9: # line 10:
      13:  # line 13:
      return x::Float64
  end::Float64


In [11]:
# LLVM bitcode produced by Julia compiler
# the type-stable function compiles a much smaller amount of code

@code_llvm Trunc(2.5)


define %jl_value_t* @julia_Trunc_71394(double) #0 {
top:
  %1 = fcmp uge double %0, 0.000000e+00
  br i1 %1, label %L, label %if

L:                                                ; preds = %top
  %2 = call %jl_value_t*** @jl_get_ptls_states() #1
  %3 = bitcast %jl_value_t*** %2 to i8*
  %4 = call %jl_value_t* @jl_gc_pool_alloc(i8* %3, i32 1384, i32 16)
  %5 = getelementptr inbounds %jl_value_t, %jl_value_t* %4, i64 -1, i32 0
  store %jl_value_t* inttoptr (i64 4642611376 to %jl_value_t*), %jl_value_t** %5, align 8
  %6 = bitcast %jl_value_t* %4 to double*
  store double %0, double* %6, align 8
  ret %jl_value_t* %4

if:                                               ; preds = %top
  ret %jl_value_t* inttoptr (i64 4642226272 to %jl_value_t*)
}


In [12]:
@code_llvm Trunc_fixed(2.5)


define double @julia_Trunc_fixed_71407(double) #0 {
top:
  %.inv = fcmp olt double %0, 0.000000e+00
  %. = select i1 %.inv, double 0.000000e+00, double %0
  ret double %.
}


In [13]:
# assembly instructions directly run on the computer's processor

@code_native Trunc(2.5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[4]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	pushq	%rax
	movabsq	$4642226272, %rbx       ## imm = 0x114B2C060
Source line: 2
	xorps	%xmm1, %xmm1
	ucomisd	%xmm0, %xmm1
	ja	L94
	movabsq	$jl_get_ptls_states_fast, %rax
	movsd	%xmm0, -16(%rbp)
	callq	*%rax
Source line: 5
	movabsq	$jl_gc_pool_alloc, %rcx
	movl	$1384, %esi             ## imm = 0x568
	movl	$16, %edx
	movq	%rax, %rdi
	callq	*%rcx
	addq	$385104, %rbx           ## imm = 0x5E050
	movq	%rbx, -8(%rax)
	movsd	-16(%rbp), %xmm0        ## xmm0 = mem[0],zero
	movsd	%xmm0, (%rax)
	addq	$8, %rsp
	popq	%rbx
	popq	%rbp
	retq
Source line: 3
L94:
	movq	%rbx, %rax
	addq	$8, %rsp
	popq	%rbx
	popq	%rbp
	retq
	nopl	(%rax,%rax)


In [14]:
@code_native Trunc_fixed(2.5)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	xorpd	%xmm1, %xmm1
	maxsd	%xmm0, %xmm1
Source line: 13
	movapd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [15]:
function sumsqrtn(n)
    # Int64
    r = 0
    for i = 1:n
        # return Float64
        r = r + sqrt(i)
    end
    return r
end
@code_warntype sumsqrtn(5)

Variables:
  #self#::#sumsqrtn
  n::Int64
  r::Any
  #temp#@_4::Int64
  i::Int64
  #temp#@_6::LambdaInfo
  #temp#@_7::Float64

Body:
  begin 
      r::Any = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#@_4::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(2),1)))::Bool)) goto 30
      SSAValue(3) = #temp#@_4::Int64
      SSAValue(4) = (Base.box)(Int64,(Base.add_int)(#temp#@_4::Int64,1))
      i::Int64 = SSAValue(3)
      #temp#@_4::Int64 = SSAValue(4) # line 4:
      unless (Core.isa)(r::Union{Float64,Int64},Float64)::Any goto 15
      #temp#@_6::LambdaInfo = LambdaInfo for +(::Float64, ::Float64)
      goto 24
      15: 
      unless (Core.isa)(r::Union{Float64,Int64},Int64)::Any goto 19
      #temp#@_6::LambdaInfo = LambdaInfo for +(::Int64, ::Float64)
      goto 24
      19: 
      goto 21
      

In [17]:
function sumsqrtn_fixed(n)
    # Float64
    r = 0.0
    for i = 1:n
        # return Float64
        r = r + sqrt(i)
    end
    return r
end
@code_warntype sumsqrtn_fixed(5)

Variables:
  #self#::#sumsqrtn_fixed
  n::Int64
  r::Float64
  #temp#::Int64
  i::Int64

Body:
  begin 
      r::Float64 = 0.0 # line 4:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(2),1)))::Bool)) goto 15
      SSAValue(3) = #temp#::Int64
      SSAValue(4) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      i::Int64 = SSAValue(3)
      #temp#::Int64 = SSAValue(4) # line 6:
      r::Float64 = (Base.box)(Base.Float64,(Base.add_float)(r::Float64,(Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)((Base.box)(Float64,(Base.sitofp)(Float64,i::Int64))))::Float64))
      13: 
      goto 5
      15:  # line 8:
      return r::Float64
  end::Float64


In [18]:
@benchmark sumsqrtn(1000_000)

================ Benchmark Results ========================
     Time per evaluation: 31.51 ms [27.55 ms, 35.47 ms]
Proportion of time in GC: 7.64% [6.73%, 8.55%]
        Memory allocated: 45.78 mb
   Number of allocations: 3000000 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 3.43 s


In [19]:
@benchmark sumsqrtn_fixed(1000_000)

================ Benchmark Results ========================
     Time per evaluation: 8.82 ms [8.47 ms, 9.17 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 1.10 s


In [20]:
function string_zeros(s::AbstractString)
    x = Array(s=="Int64"?Int64:Float64, 1_000_000)
    for i in 1:length(x)
        x[i] = 0 
    end
    return x 
end
@benchmark string_zeros("Int64")

================ Benchmark Results ========================
     Time per evaluation: 24.60 ms [20.54 ms, 28.66 ms]
Proportion of time in GC: 9.88% [8.30%, 11.47%]
        Memory allocated: 22.88 mb
   Number of allocations: 999492 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 2.70 s


In [21]:
function string_zeros_fixed(s::AbstractString)
    x = Array(s=="Int64"?Int64:Float64, 1_000_000)
    return fill_zeros(x)
end

function fill_zeros(x)
    for i in 1:length(x)
        x[i] = 0 
    end
    return x 
end

@benchmark string_zeros_fixed("Int64")

================ Benchmark Results ========================
     Time per evaluation: 3.05 ms [2.17 ms, 3.93 ms]
Proportion of time in GC: 18.96% [2.56%, 35.37%]
        Memory allocated: 7.63 mb
   Number of allocations: 2 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.53 s


* Types at storage locations

In [1]:
a = Int64[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

b = Number[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

function arr_sumsqr{T <: Number}(x::Array{T})
    r = zero(0)
    for i = 1 : length(x)
        r = r + x[i]^2
    end
    return r
end

arr_sumsqr (generic function with 1 method)

In [3]:
using Benchmarks
@benchmark arr_sumsqr(a)

================ Benchmark Results ========================
     Time per evaluation: 33.08 ns [32.44 ns, 33.73 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 4901
   Number of evaluations: 213901
         R² of OLS model: 0.951
 Time spent benchmarking: 3.44 s


In [4]:
# When the array is de ned to contain a speci c concrete type, 
# the Julia runtime can store the values inline within the allocation of the array 
# since it knows the exact size of each element. 
# When the array can contain an abstract type, the actual value can be of any size. 
# Thus, when the Julia runtime creates the array, 
# it only stores the pointers to the actual values within the array. 
# The values are stored elsewhere on the heap

@benchmark arr_sumsqr(b)

================ Benchmark Results ========================
     Time per evaluation: 467.71 ns [458.91 ns, 476.51 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 5301
   Number of evaluations: 313201
         R² of OLS model: 0.950
 Time spent benchmarking: 3.96 s


-------------------------------
4 **Functions and Macros**

> Functions in Julia are **first class** entities

* Using globals

> bad programming practice

> The compiler cannot keep track of all writes to global variables; this would be akin to solving the **halting problem**. Therefore, the dataflow technique fails to perform any inference for these types of global variables

In [10]:
# global variable
p = 2

function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y ^ p
    end
    return s
end

t = rand(100_000)
@benchmark pow_array(t)

================ Benchmark Results ========================
     Time per evaluation: 9.03 ms [7.21 ms, 10.85 ms]
Proportion of time in GC: 3.64% [0.00%, 8.44%]
        Memory allocated: 4.58 mb
   Number of allocations: 300000 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 1.30 s


In [11]:
@code_warntype pow_array(t)

Variables:
  #self#::#pow_array
  x::Array{Float64,1}
  s::Any
  #temp#::Int64
  y::Float64

Body:
  begin 
      s::Any = 0.0 # line 6:
      #temp#::Int64 = $(QuoteNode(1))
      4: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)((Base.arraylen)(x::Array{Float64,1})::Int64,1)))::Bool)) goto 14
      SSAValue(2) = (Base.arrayref)(x::Array{Float64,1},#temp#::Int64)::Float64
      SSAValue(3) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      y::Float64 = SSAValue(2)
      #temp#::Int64 = SSAValue(3) # line 7:
      s::Any = (s::Any + (y::Float64 ^ Main.p)::Any)::Any
      12: 
      goto 4
      14:  # line 9:
      return s::Any
  end::Any


In [12]:
# get back performance
# or pass the global as a function argument
const p2 = 2

function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y ^ p2
    end
    return s
end

t = rand(100_000)
@benchmark pow_array2(t)

================ Benchmark Results ========================
     Time per evaluation: 105.63 μs [102.26 μs, 109.01 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.26 s


In [14]:
@code_warntype pow_array2(t)

Variables:
  #self#::#pow_array2
  x::Array{Float64,1}
  s::Float64
  #temp#::Int64
  y::Float64

Body:
  begin 
      s::Float64 = 0.0 # line 6:
      #temp#::Int64 = $(QuoteNode(1))
      4: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)((Base.arraylen)(x::Array{Float64,1})::Int64,1)))::Bool)) goto 14
      SSAValue(2) = (Base.arrayref)(x::Array{Float64,1},#temp#::Int64)::Float64
      SSAValue(3) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      y::Float64 = SSAValue(2)
      #temp#::Int64 = SSAValue(3) # line 7:
      s::Float64 = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.Math.box)(Base.Math.Float64,(Base.Math.powi_llvm)(y::Float64,(Base.box)(Int32,(Base.checked_trunc_sint)(Int32,Main.p2))))::Float64))
      12: 
      goto 4
      14:  # line 9:
      return s::Float64
  end::Float64


* Inlining

> Inlining is an optimization performed by a compiler, where the contents of a function or method is inserted directly into the body of the **caller** of that function

> Julia uses the LLVM compiler to generate **machine code**, which is finally run on the CPU. Most of the usual **compiler optimization techniques** that run on Julia code are performed by LLVM. The one major exception is inlining, which is performed by the Julia compiler itself before LLVM is invoked

> While inlining usually results in an increase in code speed, it also simultaneously increases the size of the code

>Disable inlining **-inline=no**: during **complex debugging sessions** / while running **code coverage analysis** 

In [15]:
Trunc(x) = x<0 ? zero(x) : x

function sqrt_sin(x)
    y = Trunc(x)
    return sin(sqrt(y) + 1)
end

# processed AST after the compiler has run its type inference and inlining passes
@code_typed sqrt_sin(1)

LambdaInfo for sqrt_sin(::Int64)
:(begin 
        # meta: location In[15] Trunc 1
        unless (Base.slt_int)(x,0)::Bool goto 5
        #temp# = 0
        goto 7
        5: 
        #temp# = x
        7: 
        # meta: pop location
        y = #temp# # line 5:
        return $(Expr(:invoke, LambdaInfo for sin(::Float64), :(Main.sin), :((Base.box)(Base.Float64,(Base.add_float)((Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)((Base.box)(Float64,(Base.sitofp)(Float64,y))))::Float64,(Base.box)(Float64,(Base.sitofp)(Float64,1)))))))
    end::Float64)

In [19]:
@inline function f_inline(x)
    a = x * 5
    b = a + 3
    c = a - 4
    d = b / c
end

g_inline(x) = f_inline(2 * x)

@code_typed g_inline(3)

LambdaInfo for g_inline(::Int64)
:(begin 
        # meta: location In[19] f_inline 2
        a = (Base.box)(Int64,(Base.mul_int)((Base.box)(Int64,(Base.mul_int)(2,x)),5)) # line 3:
        b = (Base.box)(Int64,(Base.add_int)(a,3)) # line 4:
        c = (Base.box)(Int64,(Base.sub_int)(a,4)) # line 5:
        SSAValue(0) = (Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,b)),(Base.box)(Float64,(Base.sitofp)(Float64,c))))
        d = SSAValue(0)
        # meta: pop location
        return SSAValue(0)
    end::Float64)

In [20]:
@code_llvm g_inline(3)


define double @julia_g_inline_71806(i64) #0 {
top:
  %1 = mul i64 %0, 10
  %2 = add i64 %1, 3
  %3 = add i64 %1, -4
  %4 = sitofp i64 %2 to double
  %5 = sitofp i64 %3 to double
  %6 = fdiv double %4, %5
  ret double %6
}


* Closures and anonymous functions

> In Julia, anonymous functions are created with the **->** operator separating the arguments from the function body. These and named functions created within the scope of another function, and referring to variables from this outer scope, are called closures

In [21]:
sqr(x) = x ^ 2
# named function
@benchmark map(sqr, rand(100_000))

================ Benchmark Results ========================
     Time per evaluation: 511.97 μs [202.79 μs, 821.15 μs]
Proportion of time in GC: 2.41% [0.00%, 10.69%]
        Memory allocated: 781.36 kb
   Number of allocations: 4 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.33 s


In [23]:
# anonymous function
# Before v0.5, using a named function is about twice as fast as using an anonymous function

@benchmark map(x -> x ^ 2,rand(100_000))

================ Benchmark Results ========================
     Time per evaluation: 389.48 μs [121.01 μs, 657.95 μs]
Proportion of time in GC: 2.29% [0.00%, 10.15%]
        Memory allocated: 781.36 kb
   Number of allocations: 4 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.31 s


* Using macros for performance

> As the compiler goes through each stage, it can write code to execute at various points along this pipeline rather than everything waiting until the end -- the runtime

> Raw source -> Parse -> Macro Expansion **macroexpand()** -> lowering **@code_lowered** -> Type Inference 
 -> Inlining -> Generated Function Expansion **code_typed** -> CodeGen **@code_llvm** -> Native Compilation **@code_native** -> Run

> The **peakflops()** Julia function will return the maximum number of floating point operations per second (**flops**) possible on the current processor.

> Example

$$p(x) = \sum_{i=0}^n a_i x^i = a_0 + a_1 x + a_2 x^2 + \dots + a_n x^n$$

In [26]:
function poly_naive(x,a...)
    # type-stable
    p = zero(x)
    
    for i = 1:length(a)
        p = p + a[i] * x ^ (i - 1)
    end
    return p
end

f_naive(x) = poly_naive(x, 1, 2, 3, 4, 5)
f_naive(3.5)

@benchmark f_naive(3.5)

================ Benchmark Results ========================
     Time per evaluation: 54.23 ns [53.19 ns, 55.28 ns]
Proportion of time in GC: 0.15% [0.00%, 0.33%]
        Memory allocated: 32.00 bytes
   Number of allocations: 2 allocations
       Number of samples: 10201
   Number of evaluations: 33352001
         R² of OLS model: 0.904
 Time spent benchmarking: 10.05 s


> floating-point exponentiation is a particularly expensive operation

> Horner method, named after 19th century British mathematician **William George Horner**, can replace the exponentiation into multiplications

$$b_n = a_n$$
$$b_{n-1} = a_{n-1} + b_n x$$
$$b_{n-2} = a_{n-2} + b_{n-1} x$$
$$\vdots$$
$$b_0 = a_0 + b_1 x = p(x)$$

In [28]:
function poly_horner(x, a...)
    b = zero(x)
    for i = length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

f_horner(x) = poly_horner(x, 1, 2, 3, 4, 5)
f_horner(3.5)

@benchmark f_horner(3.5)

================ Benchmark Results ========================
     Time per evaluation: 45.18 ns [44.32 ns, 46.04 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 32.00 bytes
   Number of allocations: 2 allocations
       Number of samples: 4701
   Number of evaluations: 176701
         R² of OLS model: 0.955
 Time spent benchmarking: 3.96 s


In [33]:
# faster method
# Improving the speed of this computation starts with realizing that the coef cients of the polynomial are constants
f_fast(x) = muladd(x,muladd(x,muladd(x,muladd(x,5,4),3),2),1)
f_fast(3.5)

@benchmark f_fast(3.5)

================ Benchmark Results ========================
     Time per evaluation: 2.91 ns [2.87 ns, 2.95 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 8901
   Number of evaluations: 9662201
         R² of OLS model: 0.957
 Time spent benchmarking: 6.66 s


In [35]:
# Metaprogramming

# macro that will produce the previous expression when given a set of polynomial coef cients
macro horner(x, p...)
    # esc is only valid in the context of an Expr returned from a macro. Prevents the
    # macro hygiene pass from turning embedded variables into gensym variables
    ex = esc(p[end])
    
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    # Expr(head::Symbol, args::Array{Any,1}, typ::Any)
    Expr(:block, :(t = $(esc(x))), ex)
end

f_horner_macro(x) = @horner(x, 1, 2, 3, 4, 5)
f_horner_macro(3.5)

@benchmark f_horner_macro(3.5)

================ Benchmark Results ========================
     Time per evaluation: 3.01 ns [2.97 ns, 3.05 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 10901
   Number of evaluations: 64991201
         R² of OLS model: 0.951
 Time spent benchmarking: 8.40 s


* Using generated functions

In [36]:
# calculate the number of cells of a multidimensional array
function prod_dim{T, N}(x::Array{T,N})
    s = 1
    for i = 1:N
        s = s * size(x, i)
    end
    return s
end

prod_dim(rand(10, 5, 5))

250

In [37]:
# the number of iterations of the loop is equal to the number of dimensions of the array,
# which is encoded as a type parameter for arrays
@generated function prod_dim_gen{T, N}(x::Array{T,N})
    ex = :(1)
    for i = 1:N
        ex = :(size(x, $i) * $ex)
    end
    return ex
end

prod_dim_gen(rand(10, 5, 5))

250

* Using named parameters

> using named parameters can cause degraded performance

> when designing high-level functions, it is still advantageous to use named parameters 
in order to create easy to use API's. Just don't use them in performance-sensitive inner loops

> the **Benchmarks** package does not support named arguments

In [38]:
# named arguments
named_param(x; y = 1, z = 1) = x^y + x^z
# regular, positional arguments
pos_param(x, y, z) = x^y + x^z

@time for i in 1:100_000; named_param(4;y = 2, z = 3); end

  0.021600 seconds (100.00 k allocations: 10.681 MB)


In [39]:
@time for i in 1:100_000; pos_param(4, 2, 3); end

  0.000880 seconds


-------------------------------
5 **Fast Numbers**

* Numbers in Julia

> Integer: as a Julia value, basic numeric types can be **boxed**. That is, when stored in memory they are prefixed with a tag that represents their type

In [45]:
Base.WORD_SIZE

# The Int type alias represents the actual integer type used by the system
typeof(Int(2))

# the default integer types are signed
bits(3)
bits(-3)

isbits(Int64)
isbits(ASCIIString)

  likely near In[45]:1
  likely near In[45]:11


false

In [46]:
myadd(x, y) = x + y

@code_native myadd(1, 2)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[46]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	(%rdi,%rsi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [49]:
typemax(Int64)
typemin(Int64)

bits(typemax(Int64))
bits(typemin(Int64))

"1000000000000000000000000000000000000000000000000000000000000000"

In [59]:
2 ^ 63 - 1
2 ^ 63
2 ^ 64

# Operations on Int128 are slower, and for BigInts they are much slower than for the default integers
big(9223372036854775807) + 1
big(2) ^ 64

18446744073709551616

> Floating points: The **IEEE 754 standard** is the universally accepted technical standard for floating point operations in computer hardware and software; The binary storage standard for 64-bit floating point numbers consists of 1 sign bit, 11 bits of exponent, and 52 bits of the mantissa (or the significand)

In [61]:
bits(2.5)
bits(-2.5)

"1100000000000100000000000000000000000000000000000000000000000000"

> Unchecked conversion

In [68]:
UInt64(UInt32(1))
UInt32(UInt64(1))

# UInt32(typemax(UInt64))

# when working with binary data, it may be acceptable to truncate 64-bit values to 32-bit values without checking
typemax(UInt64) % UInt32

LoadError: InexactError()

* Trading performance for accuracy

> **@fastmath**: loosen the constraints of IEEE floating point operations in order to achieve greater performance. This option is similar to the **-ffast-math** compiler option in clang or GCC

In [71]:
function sum_diff(x)
    n = length(x)
    d = 1 / (n - 1)
    s = zero(eltype(x))
    s = s + (x[2] - x[1]) / d
    
    for i = 2 : length(x) - 1
        s = s + (x[i+1] - x[i+1]) / (2 * d)
    end
    
    s = s + (x[n] - x[n-1]) / d
end

t= rand(100_000)
sum_diff(t)
@benchmark sum_diff(t)

================ Benchmark Results ========================
     Time per evaluation: 285.88 μs [273.88 μs, 297.89 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.28 s


In [72]:
function sum_diff_fast(x)
    n = length(x)
    d = 1 / (n - 1)
    s = zero(eltype(x))
    @fastmath s = s + (x[2] - x[1]) / d
    
    @fastmath for i = 2 : length(x) - 1
        s = s + (x[i+1] - x[i+1]) / (2 * d)
    end
    
    @fastmath s = s + (x[n] - x[n-1]) / d
end

sum_diff_fast(t)
@benchmark sum_diff_fast(t)

================ Benchmark Results ========================
     Time per evaluation: 35.27 μs [34.56 μs, 35.98 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.30 s


In [73]:
function sum_diff_fast(x)
    n = length(x)
    d = 1 / (n - 1)
    s = zero(eltype(x))
    @fastmath s = s + (x[2] - x[1]) / d
    
    # the macro rewrites the intrinsic functions with its own _fast versions
    macroexpand(:(@fastmath for i = 2 : length(x) - 1
        s = s + (x[i+1] - x[i+1]) / (2 * d)
    end))
    
    @fastmath s = s + (x[n] - x[n-1]) / d
end

sum_diff_fast(t)
@benchmark sum_diff_fast(t)

================ Benchmark Results ========================
     Time per evaluation: 115.85 μs [80.18 μs, 151.53 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 15.47 kb
   Number of allocations: 253 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.26 s


> Adding a collection of floating point values: A naive implementation—that is, adding elements from the first to the last—accumulates errors at the rate of $\mathcal{O}$($\sqrt{n}$), where n is the number of elements being summed. Julia's **sum** base uses a pairwise summation algorithm that does better by accumulating errors at $\mathcal{O}$( $\sqrt{log(n)}$) but is almost as fast. However, there exists a more complicated summation algorithm attributed to **William Kahan** whose error is bound by $\mathcal{O}$(1)

In [76]:
sum([1 1e-100 -1])

# Kahan-Babuska-Neumaier compensated summation algorithm
sum_kbn([1 1e-100 -1])

@benchmark sum([1 1e-100 -1])

================ Benchmark Results ========================
     Time per evaluation: 7.22 ns [7.12 ns, 7.31 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 9801
   Number of evaluations: 22780601
         R² of OLS model: 0.956
 Time spent benchmarking: 7.82 s


In [77]:
@benchmark sum_kbn([1 1e-100 -1])

================ Benchmark Results ========================
     Time per evaluation: 11.09 ns [10.90 ns, 11.28 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 5701
   Number of evaluations: 458401
         R² of OLS model: 0.955
 Time spent benchmarking: 4.51 s


* Subnormal numbers [Computer Science]

> very small floating point values near zero. Formally, they are numbers smaller than those that can be represented without leading zeros in the significand (for example, normal numbers)

In [87]:
set_zero_subnormals(false)
issubnormal(1.0)

issubnormal(1.0e-308)

3e-308 - 3.001e-308
issubnormal(3e-308 - 3.001e-308)

true

In [88]:
set_zero_subnormals(true)

3e-308 - 3.001e-308

3e-308 == 3.001e-308

get_zero_subnormals()

true

In [93]:
function timestep(b, a, dt)
    n = length(b)
    b[1] = 1
    two = eltype(b)(2)
    for i = 2:n-1
        b[i] = a[i] + (a[i-1] - two * a[i] + a[i+1]) * dt
    end
    b[n] = 0 
end
   
function heatflow(a, nstep)
    b = similar(a)
    o = eltype(a)(0.1)
    for t=1:div(nstep,2)
        timestep(b,a,o)
        timestep(a,b,o)
    end
end

a = rand(10000)
set_zero_subnormals(false)
@benchmark heatflow(a, 1000)

================ Benchmark Results ========================
     Time per evaluation: 18.65 ms [18.00 ms, 19.30 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 78.20 kb
   Number of allocations: 2 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 2.15 s


In [94]:
set_zero_subnormals(true)
@benchmark heatflow(a, 1000)

================ Benchmark Results ========================
     Time per evaluation: 18.62 ms [17.56 ms, 19.68 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 78.20 kb
   Number of allocations: 2 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 2.13 s


-------------------------------
6 **Fast Arrays**

* Array internals and storage

> **Single Instruction Multiple Data** (SIMD)

> Column-wise storage: similar to **MATLAB** and **Fortran**. In Julia, the array is stored with the last dimension first.

In [11]:
function col_iter(x)
    s = zero(eltype(x))
    for i = 1:size(x, 2)
        for j = 1:size(x, 1)
            s = s + x[j, i]^2
            x[j, i] = s
        end
    end
    return x
end

function row_iter(x)
    s= zero(eltype(x))
    for i = 1:size(x, 1)
        for j = 1:size(x, 2)
            s = s + x[i, j]^2
            x[i, j] = s
        end
    end
    return x
end

row_iter (generic function with 1 method)

In [13]:
a = rand(1000, 1000)
using Benchmarks
@benchmark col_iter(a)

================ Benchmark Results ========================
     Time per evaluation: 1.36 ms [1.22 ms, 1.49 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.36 s


In [14]:
@benchmark row_iter(a)

================ Benchmark Results ========================
     Time per evaluation: 3.80 ms [3.58 ms, 4.02 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.63 s


* Bounds checking

> While this cost is reasonably small and is usually a good trade-off for safety, in some situations, where it can be guaranteed that the array bounds are never crossed, it may be worthwhile to remove these checks

> When the Julia environment is started with **–check-bounds=yes**, all @inbounds annotations in code are ignored, and bound checks are mandatorily performed

> When the Julia runtime is started with **–check-bounds=no**, no bound checking is done at all. This is equivalent to annotating all array access with the @inbounds macro

In [22]:
function prefix_bounds(a, b)
    for i = 2:size(a, 1)
        a[i] = b[i-1] + b[i]
    end
    return a
end

function prefix_inbounds(a, b)
    @inbounds for i = 2:size(a, 1)
        a[i] = b[i-1] + b[i]
    end
    return a
end

x = ones(10)
y = rand(10)

@benchmark prefix_bounds(x, y)

================ Benchmark Results ========================
     Time per evaluation: 14.09 ns [13.88 ns, 14.31 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 7901
   Number of evaluations: 3726101
         R² of OLS model: 0.951
 Time spent benchmarking: 5.84 s


In [23]:
@benchmark prefix_inbounds(x, y)

================ Benchmark Results ========================
     Time per evaluation: 9.41 ns [9.28 ns, 9.54 ns]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 9301
   Number of evaluations: 14145701
         R² of OLS model: 0.951
 Time spent benchmarking: 6.98 s


* Allocation and In-place operations

> Garbage collection(GC)

In [27]:
function xpow(x)
    return [x x^2 x^3 x^4]
end

function xpow_loop(n)
    s = 0
    for i = 1:n
        s = s + xpow(i)[2]
    end
    return s
end

@benchmark xpow_loop(1000_000)

@time xpow_loop(1000_000)

  0.099535 seconds (5.00 M allocations: 167.839 MB, 12.46% gc time)


333333833333500000

In [28]:
# The exclamation mark(!) denotes that this function takes an output variable that mutates as an argument.
# makes an in-place operation
function xpow!(result::Array{Int, 1}, x)
    @assert length(result) == 4
    result[1] = x
    result[2] = x^2
    result[3] = x^3
    result[4] = x^4
end

# preallocate a single array to hold the result for each iteration
function xpow_loop_noalloc(n)
    r = [0, 0, 0, 0]
    s = 0
    for i = 1:n
        xpow!(r, i)
        s = s + r[2]
    end
    return s
end

@benchmark xpow_loop_noalloc(1000_000)

@time xpow_loop_noalloc(1000_000)

  0.013278 seconds (6 allocations: 288 bytes)


333333833333500000

In [29]:
a = rand(1000_000)
@benchmark sort(a)

================ Benchmark Results ========================
     Time per evaluation: 84.00 ms [81.80 ms, 86.19 ms]
Proportion of time in GC: 1.19% [0.16%, 2.22%]
        Memory allocated: 7.63 mb
   Number of allocations: 4 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 8.90 s


In [30]:
@benchmark sort!(a)

================ Benchmark Results ========================
     Time per evaluation: 15.78 ms [11.84 ms, 19.72 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 1.94 s


* Array Views

> **sub()**: returns a new array that is actually a view into the original array. Creating a SubArray is very fast, much faster than creating a sliced copy. Accessing a **SubArray** can be slower than accessing a regular dense array, but Julia's standard library has some extremely well-tuned code for this purpose

> In Julia v0.5, **view()** is used instead of sub() 

In [34]:
function sum_vector(x::Array{Float64, 1})
    s = 0.0
    for i = 1:length(x)
        s = s + x[i]
    end
    return s
end

# in Julia, array slices create a copy of the slice
function sum_cols_matrix(x::Array{Float64, 2})
    num_cols = size(x, 2)
    s = zeros(num_cols)
    for i = 1:num_cols
        s[i] = sum_vector(x[:, i])
    end
    return s
end

@benchmark sum_cols_matrix(rand(1000, 1000))

================ Benchmark Results ========================
     Time per evaluation: 4.82 ms [3.53 ms, 6.12 ms]
Proportion of time in GC: 14.01% [2.08%, 25.93%]
        Memory allocated: 7.78 mb
   Number of allocations: 2490 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.71 s


In [37]:
# loosen the parameter type
function sum_vector(x::AbstractArray)
    s = 0.0
    for i = 1:length(x)
        s = s + x[i]
    end
    return s
end

# in Julia, array slices create a copy of the slice
function sum_cols_matrix_views(x::Array{Float64, 2})
    num_cols = size(x, 2)
    num_rows = size(x, 1)
    s = zeros(num_cols)
    for i = 1:num_cols
        s[i] = sum_vector(view(x, 1:num_rows, i))
    end
    return s
end

@benchmark sum_cols_matrix_views(rand(1000, 1000))

================ Benchmark Results ========================
     Time per evaluation: 1.85 ms [1.24 ms, 2.46 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 70.44 kb
   Number of allocations: 1001 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.40 s


* SIMD parallelization

> A typical SIMD-enabled processor, however, can add maybe eight numbers in one CPU cycle

> rewriting code to operate on parts of the array in parallel can get complex quickly

In [38]:
function sum_vector!(x, y, z)
    n = length(x)
    for i = 1:n
        x[i] =y[i] + z[i]
    end
end

# Bounds checking is disabled for SIMD loops (Bound checking can cause branches due to exceptional conditions.)
function sum_vector_simd!(x, y, z)
    n = length(x)
    @inbounds @simd for i = 1:n
        x[i] =y[i] + z[i]
    end
end

@benchmark sum_vector!(zeros(Float32, 1000_000), rand(Float32, 1000_000), rand(Float32, 1000_000))

================ Benchmark Results ========================
     Time per evaluation: 1.82 ms [1.27 ms, 2.38 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.42 s


In [40]:
@benchmark sum_vector_simd!(zeros(Float32, 1000_000), rand(Float32, 1000_000), rand(Float32, 1000_000))

================ Benchmark Results ========================
     Time per evaluation: 1.08 ms [1.01 ms, 1.15 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.33 s


> check whether the compiler successfully vectorized your code

> the keywords to look for in the output are sections prefixed with vector and vectorized operations that look similar to <n $*$ float$>$

> **performance gains** are indeed coming from an **automatic vectorization** of our sequential code

In [41]:
@code_llvm sum_vector_simd!(zeros(Float32, 1000_000), rand(Float32, 1000_000), rand(Float32, 1000_000))


define void @"julia_sum_vector_simd!_72109"(%jl_value_t*, %jl_value_t*, %jl_value_t*) #0 {
top:
  %3 = getelementptr inbounds %jl_value_t, %jl_value_t* %0, i64 1
  %4 = bitcast %jl_value_t* %3 to i64*
  %5 = load i64, i64* %4, align 8
  %6 = icmp sgt i64 %5, 0
  %7 = select i1 %6, i64 %5, i64 0
  %8 = call { i64, i1 } @llvm.ssub.with.overflow.i64(i64 %7, i64 1)
  %9 = extractvalue { i64, i1 } %8, 1
  br i1 %9, label %fail.split, label %top.top.split_crit_edge

top.top.split_crit_edge:                          ; preds = %top
  %10 = extractvalue { i64, i1 } %8, 0
  %11 = call { i64, i1 } @llvm.sadd.with.overflow.i64(i64 %10, i64 1)
  %12 = extractvalue { i64, i1 } %11, 1
  br i1 %12, label %top.split.split.us, label %top.split.top.split.split_crit_edge

top.split.top.split.split_crit_edge:              ; preds = %top.top.split_crit_edge
  %13 = extractvalue { i64, i1 } %11, 0
  %14 = icmp slt i64 %13, 1
  br i1 %14, label %L9, label %if13.lr.ph

top.split.split.us:                     

* Yeppp! for fast vector operations

> **transcendental functions** (log, sin, cos, exp and sumabs)

> Yeppp! software suite can be considered state-of-the-art. Primarily written at **Georgia Institute of Technology** by **Marat Dukhan**, Yeppp! provides optimized implementations of modern processors of these functions, which are much faster compared to the implementations in system libraries

In [45]:
a = rand(100_000)
@benchmark log(a)

================ Benchmark Results ========================
     Time per evaluation: 2.47 ms [1.46 ms, 3.48 ms]
Proportion of time in GC: 1.70% [0.00%, 7.77%]
        Memory allocated: 781.36 kb
   Number of allocations: 4 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.53 s


In [46]:
using Yeppp
@benchmark Yeppp.log(a)

================ Benchmark Results ========================
     Time per evaluation: 456.86 μs [199.63 μs, 714.09 μs]
Proportion of time in GC: 2.26% [0.00%, 10.02%]
        Memory allocated: 781.33 kb
   Number of allocations: 2 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.29 s


In [48]:
@benchmark Yeppp.log!(a)

================ Benchmark Results ========================
     Time per evaluation: 218.25 μs [185.63 μs, 250.86 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.27 s


* Writing generic library functions using arrays

> linear indexing

> cartesian indexing

In [53]:
function mysum_linear(a::AbstractArray)
    s = zero(eltype(a))
    for i = 1:length(a)
        s = s + a[i]
    end
    return s
end

mysum_linear(1:1000_000)
mysum_linear(reshape(1:1000_000, 100, 100, 100))
mysum_linear(reshape(1:1000_000, 1000, 1000))
mysum_linear(view(reshape(1:1000_000, 1000, 1000), 1:500, 1:500))

@benchmark mysum_linear(reshape(1:1000_000, 1000, 1000))

================ Benchmark Results ========================
     Time per evaluation: 1.46 ms [1.42 ms, 1.50 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.37 s


In [54]:
# calling the same function on a subarray is significantly slower than calling it on a regular dense array
@benchmark mysum_linear(view(reshape(1:1000_000, 1000, 1000), 1:500, 1:500))

================ Benchmark Results ========================
     Time per evaluation: 3.28 ms [3.00 ms, 3.56 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.58 s


> write generic functions that can be performant with different kinds of arrays. The simplest option is to directly iterate the array rather than iterating its indices. The iterator for each kind of array will choose the most optimal strategy for high performance.

In [55]:
function mysum_in(a::AbstractArray)
    s = zero(eltype(a))
    for i in a
        s = s + i
    end
    return s
end

@benchmark mysum_in(view(reshape(1:1000_000, 1000, 1000), 1:500, 1:500))

================ Benchmark Results ========================
     Time per evaluation: 569.41 μs [509.62 μs, 629.19 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.29 s


In [56]:
function mysum_eachindex(a::AbstractArray)
    s = zero(eltype(a))
    for i in eachindex(a)
        s = s + a[i]
    end
    return s
end

@benchmark mysum_eachindex(view(reshape(1:1000_000, 1000, 1000), 1:500, 1:500))

================ Benchmark Results ========================
     Time per evaluation: 558.86 μs [503.86 μs, 613.86 μs]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.28 s


-------------------------------
7 **Beyond the single processor**

* Parallelism in Julia

> The communication between Julia processes is **one-sided** in the sense of there being a **master process** that accepts the user's inputs and controls all the other processes. **Starting a cluster**, therefore, involves:

> either a command-line switch while starting the master Julia process(julia -p n)

> or calling methods from **REPL**

In [1]:
procs()

1-element Array{Int64,1}:
 1

In [2]:
# launch multiple Julia processes on the same machine
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [3]:
procs()

3-element Array{Int64,1}:
 1
 2
 3

In [ ]:
# start processes on other hosts by providing the hostname
# by default, uses Secure Shell (SSH) to connect to and start Julia processes on remote machines
addprocs(["10.9.2.1","10.9.2.2"])

ssh: connect to host 10.9.2.1 port 22: Connection refused


> Communication between Julia processes:

> **remote references** a reference to data residing on a different Julia process. Thereby, values can be retrieved from (or written to) such a reference

> **remote calls** a request to execute a function on a particular node. Such a call is **asynchronous** in that a remote calls finishes immediately, returning the **RemoteRef** object, which is a reference to its result. The arguments to **remotecall** are the function name, the process number to execute the function in, and the arguments to this function. The caller, then, has the option to **wait()** on the reference until the call completes and then **fetch()** the result into its own process

In [4]:
a = remotecall(sqrt, 2, 4.0)

Future(2,1,4,Nullable{Any}())

In [5]:
wait(a)

Future(2,1,4,Nullable{Any}())

In [6]:
fetch(a)

2.0

In [7]:
remotecall_fetch(sqrt, 3, 4.0)

2.0

* Programming parallel tasks

> **@everywhere**: run the same code in all the processes in the cluster

> **@spawn**: run a function in a remote process without having to specify the remote node or having to work through ambiguous syntax

In [9]:
@everywhere using Distributions

In [12]:
@everywhere rand(Normal())

In [13]:
a = @spawn randn(5, 5)^2
fetch(a)

5×5 Array{Float64,2}:
  2.00928   -3.41641    0.00519312  -0.712963  -1.2409  
 -0.564294  -0.58687   -0.63263      0.153521  -0.102204
 -6.47075    6.26515   -1.94546      3.45172    0.184656
  0.742266  -2.63888    0.335607    -0.384263   0.918937
  0.259083   0.491989   0.832052    -1.13948    4.04694 

In [14]:
# at current node
b = rand(5, 5)
# copied over to the remote node
a = @spawn b ^ 2

# Even though the two code extracts look similar, they have very different performance characteristics
fetch(a)

5×5 Array{Float64,2}:
 1.84477  1.92049   1.59153  1.86899  1.22972 
 1.02286  0.938224  0.82305  1.41715  0.539441
 2.37387  2.34766   1.95303  2.3717   1.52201 
 1.6465   1.40026   1.44954  1.41447  1.0399  
 1.63774  1.5834    1.35251  1.82918  0.929488

In [15]:
using Benchmarks
function serial_add()
    s = 0.0
    for i = 1:1000_000
        s = s + randn()
    end
    return s
end

# reduction
function parallel_add()
    return @parallel (+) for i = 1:1000_000
        randn()
    end
end

@benchmark serial_add()

================ Benchmark Results ========================
     Time per evaluation: 6.47 ms [5.75 ms, 7.19 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 0.00 bytes
   Number of allocations: 0 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 0.95 s


In [16]:
@benchmark parallel_add()

================ Benchmark Results ========================
     Time per evaluation: 4.59 ms [3.91 ms, 5.27 ms]
Proportion of time in GC: 0.00% [0.00%, 0.00%]
        Memory allocated: 22.50 kb
   Number of allocations: 281 allocations
       Number of samples: 100
   Number of evaluations: 100
 Time spent benchmarking: 1.56 s


In [17]:
x = [rand(100, 100) for _ in 1:10]

@benchmark map(svd, x)

================ Benchmark Results ========================
     Time per evaluation: 250.78 ms [221.10 ms, 280.46 ms]
Proportion of time in GC: 0.14% [0.00%, 0.45%]
        Memory allocated: 5.47 mb
   Number of allocations: 242 allocations
       Number of samples: 27
   Number of evaluations: 27
 Time spent benchmarking: 7.53 s


In [18]:
@benchmark pmap(svd, x)

================ Benchmark Results ========================
     Time per evaluation: 153.63 ms [120.94 ms, 186.32 ms]
Proportion of time in GC: 0.14% [0.00%, 0.67%]
        Memory allocated: 1.63 mb
   Number of allocations: 2023 allocations
       Number of samples: 48
   Number of evaluations: 48
 Time spent benchmarking: 10.02 s


* Shared memory arrays

> **DistributedArrays**: a fully generic solution that scales across many networked hosts in order to work on data that cannot fit in the memory of a single machine

In [19]:
using DistributedArrays

S = SharedArray(Float64, (100, 100, 5), pids=[2,3])

100×100×5 SharedArray{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.